In [26]:
from Bio import SeqIO, Entrez
import os
from urllib.error import HTTPError
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from itertools import permutations, product
from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score
import tqdm
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split, cross_val_score

from numpy import mean
from numpy import std
import pickle
from os import path
from sklearn.model_selection import cross_val_score
from warnings import simplefilter
from collections import OrderedDict
from sklearn.metrics import accuracy_score, auc, confusion_matrix, balanced_accuracy_score, precision_recall_curve, auc, roc_curve, roc_auc_score

from concurrent.futures import ThreadPoolExecutor, as_completed
import matplotlib.pyplot as plt

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action='ignore', category=FutureWarning)

if (os.path.abspath('').split('/')[-1] == 'project'):
    %cd utils
elif (os.path.abspath('').split('/')[-1] == 'validate'):
    %cd ../utils

import query_utils
import model_utils
import validation_utils
import data_utils

print(os.path.abspath(''))

if (os.path.abspath('').split('/')[-1] == 'utils'):
    %cd ..

/Users/benjaminli/Documents/coding/scires/project/utils
/Users/benjaminli/Documents/coding/scires/project/utils
/Users/benjaminli/Documents/coding/scires/project


In [27]:
best_gradBoost = pickle.load(open('models/curr_models/gradBoost.pkl', 'rb'))
best_xgBoost = pickle.load(open('models/curr_models/xgBoost.pkl', 'rb'))
nardus_model = pickle.load(open('models/nardus_testing/gradBoost.pkl', 'rb'))
gridsearch = pickle.load(open('models/nardus_gridsearch.pkl', 'rb'))

kmer = 4

s = product('acgt',repeat = kmer)
permset = set(["".join(x) for x in list(s)])

preds = []
reals = []
for ind, file in enumerate(os.listdir("data/virome_contigs")):
    fasta_sequences = SeqIO.parse(open(f"data/virome_contigs/{file}"),'fasta')

    fastas = [x for x in fasta_sequences]
    for fasta in fastas:
        name, sequence = fasta.id, str(fasta.seq)
        X_info = sequence.lower()

        oDict = data_utils.assign_kmers_to_dict(X_info, permset, kmer) # convert ordereddict to pandas dataframe

        kmer_df = pd.DataFrame()

        for i in oDict:
            kmer_df.at[0, i]=oDict[i]

        cols_when_model_builds = best_gradBoost.feature_names_in_
        kmer_df=kmer_df[cols_when_model_builds]
        
        gradboost = best_gradBoost.predict_proba(kmer_df)[:,1]
        
        cols_when_model_builds = best_xgBoost.get_booster().feature_names
        kmer_df=kmer_df[cols_when_model_builds]

        xgpred = best_xgBoost.predict_proba(kmer_df)[:,1]

        cols_when_model_builds = nardus_model.feature_names_in_
        kmer_df=kmer_df[cols_when_model_builds]
        
        narduspred = nardus_model.predict_proba(kmer_df)[:,1]

        cols_when_model_builds = gridsearch.feature_names_in_
        kmer_df=kmer_df[cols_when_model_builds]
        
        gridsearchpred = gridsearch.predict_proba(kmer_df)[:,1]

        preds.append([gradboost, xgpred, narduspred, gridsearchpred])

preds = np.array(preds)

In [28]:
for i in preds:
    print(i)

[[0.99472879]
 [0.17953709]
 [0.82483129]
 [0.97802307]]
[[0.99472879]
 [0.17953709]
 [0.82483129]
 [0.97802307]]
[[0.99472879]
 [0.17953709]
 [0.82483129]
 [0.97802307]]
[[0.81636603]
 [0.86654699]
 [0.9999348 ]
 [0.80271019]]
[[0.89563144]
 [0.62301123]
 [0.9107293 ]
 [0.7570221 ]]
[[0.9882871 ]
 [0.15187934]
 [0.91997418]
 [0.80313219]]
[[0.84290229]
 [0.65225333]
 [0.98056006]
 [0.93460681]]
[[0.99974911]
 [0.52138221]
 [0.98683607]
 [0.96909537]]
[[0.86454886]
 [0.31796557]
 [0.98473874]
 [0.96456631]]
[[0.94852308]
 [0.38062403]
 [0.99978103]
 [0.85259525]]
[[0.99530284]
 [0.53400129]
 [0.99972195]
 [0.96519673]]
[[0.98089831]
 [0.47641757]
 [0.85380748]
 [0.98208681]]
[[0.99932044]
 [0.23872426]
 [0.74158221]
 [0.96831846]]
[[0.4596301 ]
 [0.54573119]
 [0.86942593]
 [0.91373088]]
[[0.99825051]
 [0.67055488]
 [0.99975576]
 [0.98101958]]
[[0.99994481]
 [0.45912325]
 [0.94711276]
 [0.7307025 ]]
[[0.98992632]
 [0.69863379]
 [0.77338788]
 [0.99592329]]
[[0.97879981]
 [0.46988928]
 [0